# Imports

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# global imports
import os
from google.colab import drive


In [3]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import json
import re
import nltk
import string

In [4]:
! pip install contractions
!pip install -U spacy
!python -m spacy download en_core_web_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.7/110.7 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 2.5 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [5]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [6]:
import spacy
# import ktrain
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.tokenizer import Tokenizer
from spacy.lang.en import English
import re
from sklearn.feature_extraction.text import CountVectorizer
#from ktrain.text import StandardTextPreprocessor
import contractions
from gensim import corpora
from gensim.models import LdaModel
from gensim.parsing.preprocessing import preprocess_string
# getting stop_words from nltk and initializing a lemmatizer
stop_words = set(nltk.corpus.stopwords.words('english'))
lemmatizer = nltk.stem.WordNetLemmatizer()
from sklearn.cluster import KMeans
import time
import pickle
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize


# Data Cleaning

In [7]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader, Dataset
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
# import cupy as cp
import pandas as pd
import xml.etree.ElementTree as ET
from google.colab import drive
from sklearn.compose import make_column_transformer
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

In [8]:
sentiment_words = [
    "couldn't", "shouldn't", "wasn't", "doesn't", "wouldn't",
    "nor", "won't", "mustn't", "isn't", "weren't", "haven't",
    "hadn't", "doesn't", "wasn't", "aren't", "hadn't", "shan't",
    "about", "don't", "shouldn't", "didn't", "wouldn't", "needn't",
    "mightn't", "haven't", "not", "no", "very"
]

def clean_text(text):
    # Remove URLs
    text = re.sub(r'http\S+', '', text)

    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))

    # Convert to lowercase
    text = text.lower()

    # Tokenize text
    tokens = word_tokenize(text)

    # Remove stopwords
    stop_words = set(stopwords.words('english'))

    # Remove sentiment affecting words
    for i in sentiment_words:
      if i in stop_words:
        stop_words.remove(i)

    filtered_tokens = [word for word in tokens if word not in stop_words]

    # Join tokens back into string
    clean_text = ' '.join(filtered_tokens)

    return clean_text


def clean_aspect(aspect):
    # Convert to lowercase
    aspect = aspect.lower()

    # Remove punctuation
    aspect = aspect.translate(str.maketrans('', '', string.punctuation))

    return aspect

In [9]:
# load the dataset with labels

drive_path = '/content/drive/MyDrive/'
dataset_path = drive_path + 'Restaurants_Train_v2.xml'
dataset = []

f = ET.parse(dataset_path)
for i in f.getroot():
  for j in i.find("aspectCategories"):
    dataset.append({
        "input_text": clean_text(i.find("text").text),
        "aspect": clean_aspect(j.attrib['category']),
        "polarity": j.attrib['polarity']
    })

bgw_vectorizer = CountVectorizer(binary=False, stop_words='english', ngram_range=(1,4))
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1,1))
train_dataset, test_dataset = train_test_split(dataset, test_size=0.2, random_state=42)

In [10]:
df = pd.DataFrame(train_dataset)

In [11]:
df.head()

,input_text,aspect,polarity
0,decor nice casual fine dining,ambience,conflict
1,find returning quite often,anecdotesmiscellaneous,positive
2,seating trying get maximum amount people resta...,ambience,negative
3,like took leftover chicken poured oil sprinkle...,food,negative
4,finest latin women youll see life hang jimmys,ambience,positive


In [12]:
df['input_text'][4]

'finest latin women youll see life hang jimmys'

# Getting aspect nouns



In [13]:
# Load spaCy model
nlp = spacy.load("en_core_web_lg")


# Define a function to extract nouns (aspects) from a review
def extract_nouns_from_review(review, nlp):
    # Process the review using spaCy pipeline
    doc = nlp(review)
    # Extract nouns from the review
    nouns = [token.text for token in doc if token.pos_ == "NOUN"]
    return list(set(nouns))

# Extract nouns (aspects) for each review and add them as a new column to the DataFrame
df['extracted_nouns'] = df['input_text'].apply(lambda review: extract_nouns_from_review(review, nlp))

In [14]:
df.head()

,input_text,aspect,polarity,extracted_nouns
0,decor nice casual fine dining,ambience,conflict,"[dining, decor]"
1,find returning quite often,anecdotesmiscellaneous,positive,[]
2,seating trying get maximum amount people resta...,ambience,negative,"[neighbor, people, amount, seating]"
3,like took leftover chicken poured oil sprinkle...,food,negative,"[red, chicken, oil, powder, sauce, pepper]"
4,finest latin women youll see life hang jimmys,ambience,positive,"[women, life]"


In [15]:
df.shape

(2970, 4)

In [16]:
df = df[~(df['input_text']=='')] #removing blank reviews

In [17]:
df = df[df['input_text'].apply(lambda x: len(nltk.word_tokenize(x)))>1] #removing single worded reviews

In [18]:
df.shape

(2914, 4)

In [19]:
df.to_csv(drive_path+'semeval_cleaned.csv', index=False)

In [20]:
df=pd.read_csv(drive_path+'semeval_cleaned.csv')

In [21]:
df.shape

(2914, 4)

# Getting aspect categories

# BERT Topic Modelling

In [22]:
df=pd.read_csv(drive_path+'semeval_cleaned.csv')

import warnings
warnings.filterwarnings("ignore")

In [23]:
!pip install bertopic

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.1/154.1 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 84.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.3/163.3 kB 24.7 MB/s eta 0:00:00
  Using cached Cython-0.29.37-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.manylinux_2_24_x86_64.whl (1.9 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 7.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (7

In [24]:
%%time
from bertopic import BERTopic
model = BERTopic(verbose=True,embedding_model='paraphrase-MiniLM-L3-v2', min_topic_size= 4)
headline_topics, _ = model.fit_transform(df.input_text)

2024-04-17 02:58:20,292 - BERTopic - Embedding - Transforming documents to embeddings.


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.04k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/92 [00:00<?, ?it/s]

2024-04-17 02:58:32,694 - BERTopic - Embedding - Completed ✓
2024-04-17 02:58:32,695 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-04-17 02:58:53,937 - BERTopic - Dimensionality - Completed ✓
2024-04-17 02:58:53,939 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-04-17 02:58:54,099 - BERTopic - Cluster - Completed ✓
2024-04-17 02:58:54,113 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-04-17 02:58:54,385 - BERTopic - Representation - Completed ✓


CPU times: user 52.3 s, sys: 1.75 s, total: 54.1 s
Wall time: 1min 8s


In [25]:
freq = model.get_topic_info()
print("Number of topics: {}".format( len(freq)))
freq.head()

Number of topics: 177


,Topic,Count,Name,Representation,Representative_Docs
0,-1,760,-1_restaurant_not_chinese_like,"[restaurant, not, chinese, like, one, eat, tim...",[perfect restaurant ny life style got cool des...
1,0,86,0_pizza_pizzas_tastes_soggy,"[pizza, pizzas, tastes, soggy, hes, delivery, ...","[pizza really good, pizza delicious, pizza great]"
2,1,84,1_wine_list_selection_wines,"[wine, list, selection, wines, priced, italian...","[wine service very good, wine service very goo..."
3,2,56,2_never_ive_times_disappointed,"[never, ive, times, disappointed, month, nigir...","[would never go back, ive never disappointed, ..."
4,3,47,3_reservation_seated_reservations_minutes,"[reservation, seated, reservations, minutes, t...",[even though made reservation 3pm night dinner...


In [26]:
model.visualize_barchart(top_n_topics=8)

In [27]:
model.visualize_topics()

In [28]:
model.visualize_hierarchy(top_n_topics=20)

We will take 4 main topics out of these clusters - food, service, ambience, price

# Using cosine similarity to get aspect of each review

In [29]:
import spacy
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
# Load spaCy model with GloVe vectors
#!python -m spacy download en_core_web_lg #run this for every new session
nlp = spacy.load("en_core_web_lg")

# Define aspects and their corresponding words
aspects = {
    'food': 'food delicious tasty delicious flavor meal cuisine chef portion seasoned authentic',
    'service': 'service staff friendly helpful attentive prompt',
    'ambience': 'ambience ambiance atmosphere decor decoration cozy elegant modern chic view clean dirty dusty',
    'price': 'price value affordable expensive budget cost cheap',
    # 'anecdotesmiscellaneous': 'location parking cleanliness presentation variety'

}


# Define a function to compute aspect embeddings
def compute_aspect_embeddings(aspects, nlp):
    aspect_embeddings = {}
    for aspect, words in aspects.items():
        # Tokenize and average the word vectors for each aspect
        tokens = nlp(words)
        aspect_vectors = [token.vector for token in tokens if token.has_vector]
        aspect_embeddings[aspect] = np.mean(aspect_vectors, axis=0)
    return aspect_embeddings

# Define a function to compute review embeddings
def compute_review_embedding(review, nlp):
    # Process the review using spaCy pipeline
    doc = nlp(review)
    sentence_embeddings = []
    for sent in doc.sents:
        # Compute sentence embedding by averaging word vectors
        sent_vector = np.mean([word.vector for word in sent if word.has_vector], axis=0)
        sentence_embeddings.append(sent_vector)
    # Replace new words with zeros and compute review embedding
    review_embedding = np.mean(sentence_embeddings, axis=0)
    return np.nan_to_num(review_embedding)


def compute_sentence_embedding(sentence, nlp):
    # Process the sentence using spaCy pipeline
    doc = nlp(sentence)
    # Compute sentence embedding by averaging word vectors
    sent_vector = np.mean([word.vector for word in doc if word.has_vector], axis=0)
    # Replace NaN values with zeros
    return np.nan_to_num(sent_vector)


# Compute semantic similarity between review sentences and aspect embeddings
def compute_semantic_similarity(review_sentences, aspect_embeddings):
    aspect_names = list(aspect_embeddings.keys())
    similarity_scores = []
    for sentence in review_sentences:
        sentence_vector = compute_review_embedding(sentence, nlp)
        sentence_vector = sentence_vector.reshape(1, -1)  # Reshape to match cosine_similarity function input
        sentence_similarity = []
        for aspect_name, aspect_vector in aspect_embeddings.items():
            aspect_vector = aspect_vector.reshape(1, -1)  # Reshape to match cosine_similarity function input
            similarity = cosine_similarity(sentence_vector, aspect_vector)[0][0]
            sentence_similarity.append(similarity)
        similarity_scores.append({aspect_name: score for aspect_name, score in zip(aspect_names, sentence_similarity)})
    return similarity_scores

In [30]:
# review_sentences = df['input_text'][0:100].to_list() --working fine

# review_sentences = df['input_text']

review_sentences = df['input_text']

#for each review in 'cleaned_text' and get a list of aspects

# Compute aspect embeddings
aspect_embeddings = compute_aspect_embeddings(aspects, nlp)

# Compute semantic similarity
semantic_similarity_scores = compute_semantic_similarity(review_sentences, aspect_embeddings)

# Choose the aspect with the highest similarity score for each sentence
chosen_aspects = [max(scores, key=scores.get) for scores in semantic_similarity_scores]

In [31]:
# Output the chosen aspects for each sentence
for sentence, aspect in zip(review_sentences, chosen_aspects):
    print("Sentence:", sentence)
    print("Chosen Aspect:", aspect)
    print()

Streaming output truncated to the last 5000 lines.
Chosen Aspect: food

Sentence: not imagine not rushing eat
Chosen Aspect: food

Sentence: die hard peep lover
Chosen Aspect: ambience

Sentence: live neightborhood regular
Chosen Aspect: service

Sentence: id definitely recommend little place
Chosen Aspect: ambience

Sentence: recommend getting reservation even though saw people seated without one
Chosen Aspect: service

Sentence: tried various slice joints around village far best one
Chosen Aspect: service

Sentence: ive twice
Chosen Aspect: food

Sentence: keep fingers crossed whole subway ride hoping table sit not share rice congee soup people
Chosen Aspect: food

Sentence: glass leaping lizard glass prosecco mussels everything happy
Chosen Aspect: ambience

Sentence: decor nice minimalist food simple yet very well presented cooked wine list matches food very well
Chosen Aspect: food

Sentence: appetizers ate dim sum variety foods impossible criticize food
Chosen Aspect: food

Sente

In [32]:
df['aspect_predicted'] = chosen_aspects

In [33]:
df.head()

,input_text,aspect,polarity,extracted_nouns,aspect_predicted
0,decor nice casual fine dining,ambience,conflict,"['dining', 'decor']",ambience
1,find returning quite often,anecdotesmiscellaneous,positive,[],service
2,seating trying get maximum amount people resta...,ambience,negative,"['neighbor', 'people', 'amount', 'seating']",service
3,like took leftover chicken poured oil sprinkle...,food,negative,"['red', 'chicken', 'oil', 'powder', 'sauce', '...",food
4,finest latin women youll see life hang jimmys,ambience,positive,"['women', 'life']",ambience


# Accuracy check

In [34]:
df2 = df[df['aspect'] != "anecdotesmiscellaneous"]

In [35]:
cnt = (df2['aspect'] == df2['aspect_predicted']).sum()

In [36]:
cnt

1304

In [37]:
df2.shape

(2035, 5)

In [38]:
print("Accuracy: ", cnt / len(df2))

Accuracy:  0.6407862407862408


# 65% accuracy

# Anurag's Changes

In [39]:
import ngram_nb
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
train_idx = int(.8*len(df2))
model = ngram_nb.Ngram_NB(1)

shuffled_df = df2.sample(frac=1)
df_train = shuffled_df[:train_idx]
df_test = shuffled_df[train_idx:]
model.train(df_train, text_col = "input_text", category_col = "aspect")

df_polarity_predictions = []
for category in df_test["aspect"].unique():
    df_test_category = df_test[df_test["aspect"] == category]
    y_pred = df_test_category["input_text"].apply(lambda x: model.score(x, category = category)[0])
    y_true = df_test_category["polarity"]
    print(category, accuracy_score(y_true, y_pred))
    conf_mat = confusion_matrix(y_true, y_pred, labels = y_true.unique())
    print(y_true.unique())
    print(conf_mat)
    df_test_category["polarity_pred"] = y_pred
    df_polarity_predictions.append(df_test_category)
    
df_test_category
pd.concat(df_polarity_predictions).reset_index().to_csv("ngram_nb_predictions_df.csv")

                                          input_text    aspect  polarity  \
0                      decor nice casual fine dining  ambience  conflict   
2  seating trying get maximum amount people resta...  ambience  negative   
3  like took leftover chicken poured oil sprinkle...      food  negative   
4      finest latin women youll see life hang jimmys  ambience  positive   
7         ive bunch times service always outstanding   service  positive   

                                     extracted_nouns aspect_predicted  
0                                ['dining', 'decor']         ambience  
2        ['neighbor', 'people', 'amount', 'seating']          service  
3  ['red', 'chicken', 'oil', 'powder', 'sauce', '...             food  
4                                  ['women', 'life']         ambience  
7                      ['service', 'bunch', 'times']          service  


# Fabian's Changes

In [40]:
# Bi-Directional Lstm with Attention Mechanism.

%load_ext autoreload
%autoreload 2

from models import bilstm_withattention

absa = bilstm_withattention.AspectBasedSentimentAnalysis()
absa.train(df2)

pd_test = pd.DataFrame(test_dataset)
df_test, loss, accuracy = absa.test(pd_test)
print(f"Loss: {loss}, Accuracy: {accuracy}")
print(df_test.head())
df_test.to_csv("./bilstm_withattention_res.csv", encoding='utf-8')

In [ ]:
# ATAE-LSTM

%load_ext autoreload
%autoreload 2

from models import lstm_atae

absa = lstm_atae.AspectBasedSentimentAnalysis()
absa.train(df2)

pd_test = pd.DataFrame(test_dataset)
df_test, loss, accuracy = absa.test(pd_test)
print(f"Loss: {loss}, Accuracy: {accuracy}")
print(df_test.head())
df_test.to_csv("./lstm_atae_res.csv", encoding='utf-8')

# Logistic Regression

In [44]:
%load_ext autoreload
%autoreload 2

from models import logistic

logi = logistic.Logistic()
tr = pd.DataFrame(train_dataset)
logi.train(df2)
out, report = logi.test(test_dataset)
print(report)
out.to_csv("./logistic_res.csv", encoding='utf-8')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
              precision    recall  f1-score   support

           0       0.54      0.47      0.50       163
           1       0.69      0.93      0.80       445
           2       0.50      0.01      0.02        97
           3       0.00      0.00      0.00        38

    accuracy                           0.66       743
   macro avg       0.43      0.35      0.33       743
weighted avg       0.60      0.66      0.59       743



# BERT UnCased

In [43]:
%load_ext autoreload
%autoreload 2

from models import bert_uncased

bert_uncase = bert_uncased.BERTUncased(15)
bert_uncase.train(df2)
out, report = bert_uncase.test(test_dataset)
print(report)
out.to_csv("./bert_base_res.csv", encoding='utf-8')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Starting Epoch 1
Loss:  0.9274495002870359
Starting Epoch 2
Loss:  0.7722076520777521
Starting Epoch 3
Loss:  0.6246266502938242
Starting Epoch 4
Loss:  0.49019280180137437
Starting Epoch 5
Loss:  0.38709108099571937
                                            input_text  \
0    forget rush wait noise isnt actually bad mean ...   
1        type place run old friends late raucus dinner   
2                                  dont look nose like   
3                 tuk tuk one rare choices go brooklyn   
4                great food great prices great service   
..                                                 ...   
738  like music blasted system isnt great want pay ...   
739           food always tastes fresh served promptly   
740  good atmosphere combination hottest music dres...   
741  looking around saw room full new yorkers enjoy...   
742                         importantly food excellent   

                     aspect  polarity  y_pred  
0                  ambience  conflict 

In [ ]:
# Common cell to comapre and contrast varoius above models

In [ ]:
# Load and test a few sentences from the YELP dataset